<a href="https://colab.research.google.com/github/seamusrobertmurphy/ecuador-art-trees/blob/main/Winrock_Ecuador_Wildfire_Delineation_Task_5_060225.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment Setup**

In [2]:
# @title Install Packages & Repo
!pip install -U geemap leafmap geopandas numpy session_info xarray netCDF4 matplotlib localtileserver rioxarray
# !pip install geemap==0.16.4 # sometimes we need to revert back to specific legacy version for certain tools
import ee, json, geemap, ipyleaflet, os, numpy, session_info, rioxarray, rasterio, tempfile
import pandas as pd
import xarray as xr
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Activate Earth Engine

!earthengine authenticate
#!ee.Authenticate() # deprecated in certain Colab environments
ee.Initialize(project = "murphys-deforisk")

In [4]:
# @title RAM & GPU Checks

# Check Runtime GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0: print('Not connected to a GPU')
else: print(gpu_info)

# Check Runtime-Allocated RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20: print('Not using a high-RAM runtime')
else: print('You are using a high-RAM runtime!')

/bin/bash: line 1: nvidia-smi: command not found
Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# **Import Data**


In [15]:
# @title Import AOI Data
country = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.equals("ADM0_NAME", "Ecuador"))
state = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.equals("ADM0_NAME", "Ecuador"))
centroid_ee = country.first().geometry().centroid()

ecuador_bbox_ee = country.geometry().bounds()
ecuador_coords = ecuador_bbox_ee.getInfo()['coordinates'][0] # Get the coordinates of the bounding box
min_lon, min_lat, max_lon, max_lat = (
    min([c[0] for c in ecuador_coords]),
    min([c[1] for c in ecuador_coords]),
    max([c[0] for c in ecuador_coords]),
    max([c[1] for c in ecuador_coords])
)

state_list = state.aggregate_array('ADM1_NAME').distinct().getInfo()
state_list


white = {"color": "white", "width": 0.3, "lineType": "solid", "fillColor": "00000000"}
country_label = ee.FeatureCollection([ee.Feature(country.geometry().centroid(), {'country_name': country.first().get("ADM0_NAME").getInfo()})])

Map = geemap.Map()
Map.centerObject(centroid_ee, 6)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(state.style(**white), {}, "States")
Map.addLayer(country.style(**white), {}, "Country")
Map.add_labels(state,"ADM1_NAME",font_size="7pt",font_color="black",font_family="arial") #,font_weight="bold",)
Map.add_labels(country_label, "country_name", font_size="14pt", font_color="white", font_family="arial", font_weight="bold")
Map

Map(center=[-1.4328003721031835, -78.76612016462707], controls=(WidgetControl(options=['position', 'transparen…

In [17]:
# @title Import [GFED5](https://www.globalfiredata.org/index.html) Data (FAOSTAT)
# Beta downloads including latest 2023 data: https://surfdrive.surf.nl/files/index.php/s/VPMEYinPeHtWVxn?path=%2FGFED5_Beta_monthly_emissions#

FIRES_FAO_2012_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2012.nc'
FIRES_FAO_2013_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2013.nc'
FIRES_FAO_2014_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2014.nc'
FIRES_FAO_2015_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2015.nc'
FIRES_FAO_2016_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2016.nc'
FIRES_FAO_2017_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2017.nc'
FIRES_FAO_2018_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2018.nc'
FIRES_FAO_2019_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2019.nc'
FIRES_FAO_2020_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2020.nc'
FIRES_FAO_2021_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/inputs/GFED5_Beta_monthly_2021.nc'

FIRES_FAO_2012 = xr.open_dataset(FIRES_FAO_2012_path)
FIRES_FAO_2013 = xr.open_dataset(FIRES_FAO_2013_path)
FIRES_FAO_2014 = xr.open_dataset(FIRES_FAO_2014_path)
FIRES_FAO_2015 = xr.open_dataset(FIRES_FAO_2015_path)
FIRES_FAO_2016 = xr.open_dataset(FIRES_FAO_2016_path)
FIRES_FAO_2017 = xr.open_dataset(FIRES_FAO_2017_path)
FIRES_FAO_2018 = xr.open_dataset(FIRES_FAO_2018_path)
FIRES_FAO_2019 = xr.open_dataset(FIRES_FAO_2019_path)
FIRES_FAO_2020 = xr.open_dataset(FIRES_FAO_2020_path)
FIRES_FAO_2021 = xr.open_dataset(FIRES_FAO_2021_path)

fires_fao_datasets = {
    2012: FIRES_FAO_2012,
    2013: FIRES_FAO_2013,
    2014: FIRES_FAO_2014,
    2015: FIRES_FAO_2015,
    2016: FIRES_FAO_2016,
    2017: FIRES_FAO_2017,
    2018: FIRES_FAO_2018,
    2019: FIRES_FAO_2019,
    2020: FIRES_FAO_2020,
    2021: FIRES_FAO_2021,
}

print(FIRES_FAO_2012)

<xarray.Dataset> Size: 2GB
Dimensions:         (lat: 720, lon: 1440, time: 12)
Coordinates:
  * lat             (lat) float32 3kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon             (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time            (time) datetime64[ns] 96B 2012-01-01T01:00:00 ... 2012-12...
Data variables: (12/43)
    basisregions    (lat, lon) int8 1MB ...
    grid_area       (lat, lon) float32 4MB ...
    burned_area     (time, lat, lon) float32 50MB ...
    C               (time, lat, lon) float32 50MB ...
    DM              (time, lat, lon) float32 50MB ...
    CO2             (time, lat, lon) float32 50MB ...
    ...              ...
    HCN             (time, lat, lon) float32 50MB ...
    HCOOH           (time, lat, lon) float32 50MB ...
    CH3COOH         (time, lat, lon) float32 50MB ...
    MEK             (time, lat, lon) float32 50MB ...
    CH3COCHO        (time, lat, lon) float32 50MB ...
    HOCH2CHO        (time, lat, lon) fl

# **Process Data**

In [18]:
# @title Process GFED5 Data
fires_fao_datasets_clipped = {}
for year, dataset in fires_fao_datasets.items():
       clipped_dataset = dataset.sel(lat=slice(max_lat, min_lat), lon=slice(min_lon, max_lon))
       fires_fao_datasets_clipped[year] = clipped_dataset

# Confirm resolution and grid alignment of NetCDF
print("\nChecking resolution and cell size of a clipped dataset (e.g., 2012):")
if 2012 in fires_fao_datasets_clipped:
    sample_dataset = fires_fao_datasets_clipped[2012]
    if 'lat' in sample_dataset.coords and 'lon' in sample_dataset.coords:
        lat_resolution = abs(sample_dataset.coords['lat'][1] - sample_dataset.coords['lat'][0]).values.item()
        lon_resolution = abs(sample_dataset.coords['lon'][1] - sample_dataset.coords['lon'][0]).values.item()
        print(f"Latitude Resolution: {lat_resolution} degrees")
        print(f"Longitude Resolution: {lon_resolution} degrees")
        if 'grid_area' in sample_dataset.data_vars:
            approx_cell_area_sqkm = sample_dataset['grid_area'].mean().values.item() # Use mean as grid_area might vary slightly
            approx_cell_size_km = approx_cell_area_sqkm**0.5
            print(f"Approximate Cell Size (from grid_area): {approx_cell_size_km:.4f} km")
    else: print("Latitude or Longitude coordinates not found in the dataset.")
else: print("Dataset for year 2012 not found in fires_fao_datasets_clipped.")


variables_to_summarize = ['burned_area', 'C', 'CO2', 'CH4', 'N2O'] # (MtC)
annual_summarized_results = {}
annual_burned_area_sqkm = {} # convert netcdf units to sq-m

# Calculate annual sums
for year, dataset in fires_fao_datasets_clipped.items():
    summarized_results_for_year = {}
    burned_area_fraction = dataset['burned_area']
    grid_area_sqkm = dataset['grid_area']
    burned_area_time_sqm = burned_area_fraction * grid_area_sqkm # Result is in m^2
    annual_burned_area_xr_sqm = burned_area_time_sqm.sum(dim='time')
    total_annual_burned_area_value_sqm = annual_burned_area_xr_sqm.sum(dim=['lat', 'lon']).values.item()
    total_annual_burned_area_value_sqkm = total_annual_burned_area_value_sqm / 1_000_000
    annual_burned_area_sqkm[year] = total_annual_burned_area_value_sqkm
    summarized_results_for_year['burned_area_sqkm'] = total_annual_burned_area_value_sqkm

    other_variables_to_summarize = [var for var in variables_to_summarize if var != 'burned_area']
    for var_name in other_variables_to_summarize:
        variable_data = dataset[var_name]
        annual_sum_xr = variable_data.sum(dim='time')
        total_sum_value = annual_sum_xr.sum(dim=['lat', 'lon']).values.item()
        summarized_results_for_year[var_name] = total_sum_value
    annual_summarized_results[year] = summarized_results_for_year

#print("\nSummarized annual results (2012-2021):")
#for year, results in annual_summarized_results.items():
#    print(f"\n{year}:")
#    for var_name, total_sum in results.items():
#        print(f"  {var_name}: {total_sum}")

# convert to dataframe
flattened_data = []
for year, results in annual_summarized_results.items():
    row = {'Year': year}
    row.update(results)
    flattened_data.append(row)

summarized_df = pd.DataFrame(flattened_data)
summarized_df.set_index('Year', inplace=True)
print("\nSummarized annual results DataFrame:")
display(summarized_df)


Checking resolution and cell size of a clipped dataset (e.g., 2012):
Latitude Resolution: 0.25 degrees
Longitude Resolution: 0.25 degrees
Approximate Cell Size (from grid_area): 27763.7392 km

Summarized annual results DataFrame:


,burned_area_sqkm,C,CO2,CH4,N2O
Year,,,,,
2012,5534.982656,2449.179688,8378.365234,14.512712,0.867199
2013,6956.704768,2428.158936,8293.531250,13.994334,0.860935
2014,4044.485120,1819.885132,6200.252930,11.065104,0.650832
2015,3035.209472,1672.328979,5667.381348,10.782615,0.609163
2016,8350.608896,3105.023438,10472.925781,19.097794,1.108688
2017,4460.294144,1528.231323,5229.643555,8.953658,0.545716
2018,5624.167424,2319.330566,7885.971680,14.802137,0.847276
2019,3831.185408,1475.375000,5047.305664,8.718035,0.529193
2020,7022.814208,3087.229980,10413.337891,18.709742,1.105915


# **Export Data**

In [22]:
# @title Export GFED5 Rasters
# Note: Functions below are commented out to avoid overloading my goolge drive account
variables_to_export = ['burned_area', 'C', 'CO2', 'CH4', 'N2O']
output_folder = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/outputs/'

# Loop through each year and each variable to export
#for year, dataset in fires_fao_datasets_clipped.items():
#    for var_name in variables_to_export:
#        if var_name in dataset:
#            annual_variable_data = dataset[var_name].sum(dim='time')
#            annual_variable_data = annual_variable_data.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
#            annual_variable_data = annual_variable_data.rio.write_crs("EPSG:4326")
#            annual_variable_data_reprojected = annual_variable_data.rio.reproject("EPSG:3857")
#            output_filename = f'{var_name}_{year}_EPSG3857.tif'
#            output_geotiff_path = os.path.join(output_folder, output_filename)
#            annual_variable_data_reprojected.rio.to_raster(output_geotiff_path)


# **Map Data**

In [23]:
# @title Map GFED5 Rasters
import xarray as xr
import rioxarray
import os
import tempfile
import geemap
import numpy as np

burned_area_2012_path = '/content/drive/MyDrive/Colab-Notebooks/Ecuador-Wildfire-Delineation/outputs/burned_area_2012_EPSG3857.tif'

FIRES_ESA_2021 = ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate('2021-01-01', '2021-12-31').filterBounds(country.geometry())
burnedArea_2021 = FIRES_ESA_2021.select('BurnDate')
maxBA_2021 = burnedArea_2021.max()


burned_area_2012 = rioxarray.open_rasterio(burned_area_2012_path)
burned_area_mask = burned_area_2012.where(burned_area_2012 > 0)
with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as tmpfile: temp_mask_path = tmpfile.name
burned_area_mask.rio.to_raster(temp_mask_path, nodata=np.nan)


vis_params_mask = {
    'min': 0,
    'max': 1,
    'palette': ['#00000000', '#FF0000FF'],
    'nodata': np.nan
}



Map = geemap.Map()
Map.centerObject(centroid_ee, 6)
Map.add_raster(temp_mask_path, layer_name='Burned Area Footprint 2012', vis_params=vis_params_mask)
Map

# Clean up temporary file
# os.remove(temp_mask_path)

Map(center=[-1.4328003721031848, -78.76612016462705], controls=(WidgetControl(options=['position', 'transparen…